Responder en base al modelo de datos:
- Considerando únicamente la plataforma de Netflix, ¿qué actor aparece más veces?
- Top 10 de actores participantes considerando ambas plataformas en el año actual. Se aprecia flexibilidad.
- Crear un Stored Proceadure que tome como parámetro un año y devuelva una tabla con las 5 películas con mayor duración en minutos.

In [1]:
import os
import subprocess
import sys
import json
import pandas as pd
from google.cloud import bigquery, bigquery_storage

# Instalar requisitos
def install_requirements():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    
# Leer credenciales de GCP y AWS
with open('_credentials/gcp_credentials.json') as gcp_file:
    gcp_credentials = json.load(gcp_file)

# Configuración de las credenciales de GCP
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '_credentials/gcp_credentials.json'

# Configuración de las credenciales de AWS
project_id = gcp_credentials['project_id']
dataset_id = gcp_credentials['dataset_id']

# Crear un cliente de BigQuery
client = bigquery.Client()

# Configuración de Pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

- ### 1. Considerando únicamente la plataforma de Netflix, ¿qué actor aparece más veces?

In [2]:
def question(file_path,output_csv_path):
    # Leer el archivo SQL
    with open(file_path, 'r') as file:
        query = file.read()

    # Ejecutar la consulta
    query_job = client.query(query)

    # Obtener el resultado como un DataFrame de pandas
    result = query_job.result().to_dataframe()

    # Guardar el resultado en un archivo CSV
    result.to_csv(output_csv_path, index=False)

    return result

# Path a la primer consulta
file_path = 'queries/question_01.sql'

# Path al archivo de salida   
output_csv_path = 'results/question_01_result.csv'

# Ejecutar la consulta y obtener el resultado como DataFrame
df = question(file_path,output_csv_path)

# Mostrar respuesta
print("Considerando únicamente la plataforma de Netflix, el actor con más apariciones es:")
display(df)

Considerando únicamente la plataforma de Netflix, el actor con más apariciones es:


,actor,appearances
0,Anupam Kher,43


- ### 2. Top 10 de actores participantes considerando ambas plataformas en el año actual.

In [3]:
# Path a la segunda consulta
file_path = 'queries/question_02.sql'

# Path al archivo de salida   
output_csv_path = 'results/question_02_result.csv'

# Ejecutar la consulta y obtener el resultado como DataFrame
df = question(file_path,output_csv_path)

# Verificar si el DataFrame está vacío y mostrar el mensaje adecuado
if df.empty:
    print("No hay películas en el dataset que hayan sido lanzadas en el año actual.")
else:
    print("El Top 10 de actores participantes considerando ambas plataformas en el año actual es el siguiente:")
    display(df)

No hay películas en el dataset que hayan sido lanzadas en el año actual.


#### Dado que en el año actual no hay resultados podemos realizar otra consulta que lea cuál es el último año con resultados y hacer el Top 10 basados en ese año:

In [4]:
# Path a la consulta alternativa
file_path = 'queries/question_02_alt.sql'

# Path al archivo de salida   
output_csv_path = 'results/question_02_alt_result.csv'

# Ejecutar la consulta y obtener el resultado como DataFrame
df = question(file_path, output_csv_path)

# Mostrar respuesta
print("El Top 10 de actores participantes considerando ambas plataformas en el último año (con resultados) es el siguiente:")
display(df)

El Top 10 de actores participantes considerando ambas plataformas en el último año (con resultados) es el siguiente:


,actor,release_year,appearances
0,Fortune Feimster,2021,11
1,David Spade,2021,10
2,London Hughes,2021,10
3,Grey Griffin,2021,5
4,Tress MacNeille,2021,5
5,Dee Bradley Baker,2021,4
6,Fred Tatasciore,2021,4
7,Bill Farmer,2021,4
8,Surya Ganapathy,2021,4
9,Madhuri,2021,4


- ### 3. Crear un Stored Proceadure que tome como parámetro un año y devuelva una tabla con las 5 películas con mayor duración en minutos

In [5]:
# Definir el año a analizar
year = 2021

def stored_procedure():
    file_path = 'queries/question_03_sp.sql'
    
    # Leer el archivo SQL
    with open(file_path, 'r') as file:
        query = file.read().format(year=year)

    # Ejecutar la consulta
    query_job = client.query(query)

    # Espera que el procedimiento haya terminado
    query_job.result()

    return

def question_03():
    # Path a la consulta
    file_path = 'queries/question_03_query.sql'
    
    # Path al archivo de salida   
    output_csv_path = 'results/question_03.csv'

    # Leer el archivo SQL
    with open(file_path, 'r') as file:
        query = file.read()

    # Ejecutar la consulta
    query_job = client.query(query)

    # Obtener el resultado como un DataFrame de pandas
    result = query_job.result().to_dataframe()

    # Guardar el resultado en un archivo CSV
    result.to_csv(output_csv_path, index=False)

    return result

# Ejecutar procedimiento almacenado
stored_procedure()

# Ejecutar la consulta y obtener el resultado como DataFrame
df = question_03()

# Mostrar respuesta
print(f"Las 5 películas con mayor duración en minutos del año {year} son:")
display(df)

Las 5 películas con mayor duración en minutos del año 2021 son:


,duration_minutes,title
0,273,Headspace: Unwind Your Mind
1,159,Jagame Thandhiram
2,151,Aelay
3,148,Army of the Dead
4,145,Tughlaq Durbar (Telugu)
